In [1]:
#!pip install pillow

In [3]:
import tkinter as tk
from tkinter import filedialog
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
from PIL import Image, ImageTk
from keras.models import load_model
# Kategorie
categories = {0: 'jabłko', 1: 'banan', 2: 'karambola', 3: 'gujawa', 4: 'kiwi', 5: 'mango',
              6: 'pomarańcza', 7: 'brzoskwinia', 8: 'gruszka', 9: 'persymona', 10: 'pitaja', 11: 'śliwka',
              12: 'granat', 13: 'pomidor', 14: 'melon'}

def predict_image(filepath, model):
    img_size = 150  
    img = image.load_img(filepath, target_size=(img_size, img_size))
    img_array = image.img_to_array(img)
    img_array = img_array / 255.0  
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    predictions = model.predict(img_array_expanded_dims)
    predicted_class_index = np.argmax(predictions, axis=1)
    confidence = np.max(predictions) * 100
    return categories[predicted_class_index[0]], confidence

# GUI
image_history = []
current_image_index = -1

def upload_and_predict():
    global current_image_index, model
    filepath = filedialog.askopenfilename()
    if not filepath:
        return
    result_label.config(text="Przetwarzam zdjęcie...")
    report_label.config(text="")

    result, confidence = predict_image(filepath, model)  
    image_history.append((filepath, result, confidence))
    current_image_index = len(image_history) - 1
    show_current_image()

def show_current_image():
    if current_image_index < 0 or current_image_index >= len(image_history):
        return
    filepath, result, confidence = image_history[current_image_index]
    if confidence < 97:  # Check if confidence is less than 97%
        result_text = f"Nie wiem co to za owoc :( {confidence:.2f}%"
    else:
        result_text = f"To jest {result} na {confidence:.2f}%!"
    result_label.config(text=result_text)
    img_pil = Image.open(filepath).resize((100, 100))
    img_tk = ImageTk.PhotoImage(img_pil)
    image_label.config(image=img_tk)
    image_label.image = img_tk


def prev_image():
    global current_image_index
    if current_image_index > 0:
        current_image_index -= 1
        show_current_image()

def next_image():
    global current_image_index
    if current_image_index < len(image_history) - 1:
        current_image_index += 1
        show_current_image()

def report_mistake():
    try:
        global current_image_index
        if current_image_index < 0 or current_image_index >= len(image_history):
            report_label.config(text="Nie można zgłosić błędu, brak obrazu!")
            return
        filepath, result, confidence = image_history[current_image_index]
        with open("reported_mistakes.txt", "a") as f:
            f.write(f"{filepath}, {result}, confidence: {confidence}\n")
        report_label.config(text="Błąd zgłoszony!")
        print(f"Reported: {filepath}, {result}, confidence: {confidence}") 
    except Exception as e:
        report_label.config(text="Wystąpił błąd podczas zgłaszania.")
        print(f"Error: {e}") 

# Okno
app = tk.Tk()
app.title("FRUITYFY")
app.geometry("1068x600")

# Tło
background_image = Image.open('background-light.jpg')
background_image = background_image.resize((1068, 600))
background_photo = ImageTk.PhotoImage(background_image)
background_label = tk.Label(app, image=background_photo)
background_label.place(relwidth=1, relheight=1)

# Model
model = load_model('best_model.h5')

# UI
bg_color = "#F6E6C4"
button_color = "#FFFFFF"

frame = tk.Frame(app, bg=bg_color, bd=0, highlightthickness=0)
frame.pack(pady=20)

title = tk.Label(frame, text="Sprawdźmy co to za owoc", font=("Helvetica", 20), bg=bg_color)
title.pack(pady=20)

upload_btn = tk.Button(frame, text="Wgraj zdjęcie", command=upload_and_predict, bg=button_color, font=("Helvetica", 10))
upload_btn.pack(pady=10)

prev_button = tk.Button(frame, text="Poprzednie", command=prev_image, bg=button_color, font=("Helvetica", 10))
prev_button.pack(side=tk.LEFT, padx=25)

next_button = tk.Button(frame, text="Następne", command=next_image, bg=button_color, font=("Helvetica", 10))
next_button.pack(side=tk.RIGHT, padx=25)

image_label = tk.Label(frame, bg=bg_color)
image_label.pack(pady=20)

result_label = tk.Label(frame, text="Wgraj pierwsze zdjęcie by zacząć", font=("Helvetica", 16), bg=bg_color)
result_label.pack(pady=20)

# Przycisk do zgłaszania błędów
report_button = tk.Button(frame, text="Zgłoś błąd", command=report_mistake, bg=button_color, activebackground=bg_color, font=("Roboto", 10))
report_button.pack(pady=10)
report_label = tk.Label(frame, text="", font=("Helvetica", 12), bg=bg_color)
report_label.pack(pady=5)

app.mainloop()


1/1 [==============================] - 0s 58ms/step
